# 2D Frame Analysis

In [ ]:
#@title Update and use python 3.12 for performance boosts
!pip install huggingface_hub
# !pip install numpy --upgrade


# Install python 3.12 (for faster performance)
!sudo apt-get update -y
!sudo apt-get install python3.12
!sudo apt-get install python3-pip

# Change default python3 to 3.12
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 1

In [ ]:
#@title Importing Libraries
import re
import math
import time
import importlib
import numpy as np
import sympy as sp
from google.colab import files
import matplotlib.pyplot as plt
from sympy import Matrix, lambdify
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

# For importlib to reload modules when changes are made
import frame_helpers
import constraints
importlib.reload(frame_helpers)
importlib.reload(constraints)

# From constraints.py (imported this way for lint check)
from frame_helpers import *
from constraints import N_columns, N_floors, N_nod_tot, N_par_nod, N_par_tot, N_ele_tot, N_nod_ele, N_par_ele, N_tot_bound, N_plots
from constraints import X_dist, Y_dist, width_beam, height_beam, width_column, height_column, po, theta, unit_weight, elastic_mod, N_discritizations


# For huggingface
# from google.colab import userdata
# token = userdata.get('HF_TOKEN')

In [ ]:
#@title Download the SMH dataset and move it to the current working directory
import os
import shutil
from huggingface_hub import snapshot_download

repo_id = "your_repo_id"  
repo_type = "dataset"  # Specify the repository type (model or dataset)
destination_path = "SMH"
token = "your_hugging_face_token"  # Replace with your actual Hugging Face token

if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# Download the entire dataset to the specified destination path
snapshot_download(repo_id, repo_type=repo_type, revision="main", token=token, local_dir=destination_path)

# Move the 'combos' folder to the current working directory
combos_folder_path = os.path.join(destination_path, "combos")
if os.path.exists(combos_folder_path):
    shutil.move(combos_folder_path, os.getcwd())
    print(f"Moved 'combos' folder to the current working directory.")
else:
    print("The 'combos' folder does not exist in the downloaded dataset.")


# Go to "Choose file to analyze"

In [ ]:
#@title Modularizations
%%cython


import numpy as np
cimport numpy as np
from sympy import Matrix, lambdify
import sympy as sp

def calculate_X_positions(np.ndarray[int, ndim=1] indices, int N_columns, double X_dist):
    X = ((indices % N_columns) - 1) * X_dist
    np.putmask(X, X < 0, (N_columns - 1) * X_dist)
    return X

def calculate_Y_positions(np.ndarray[int, ndim=1] indices, int N_columns, double Y_dist):
    h_assigner = np.ceil(indices / N_columns - 1)
    h_assigner[h_assigner < 1] = 0
    Y = Y_dist * h_assigner
    return Y

def calculate_element_node_indices(int N_floors, int N_columns):
    # Initialize the ele_nod array
    ele_nod = np.zeros((N_floors * (2 * N_columns - 1), 2), dtype=int)

    # Calculate the indices for the vertical and horizontal elements
    for i in range(1, N_floors + 1):
        for j in range(1, N_columns + 1):
            index = (i - 1) * (2 * N_columns - 1) + j - 1
            ele_nod[index, 0] = j + (i - 1) * N_columns
            ele_nod[index, 1] = ele_nod[index, 0] + N_columns
        for j in range(1, N_columns):
            index = (i - 1) * (2 * N_columns - 1) + N_columns + j - 1
            ele_nod[index, 0] = i * N_columns + j
            ele_nod[index, 1] = ele_nod[index, 0] + 1

    return ele_nod

def initialize_symbols(int N_par_ele):
    # Define symbolic variables
    x, h_e, beta_e = sp.symbols('x h_e beta_e')
    qe = sp.Array([sp.Symbol(f'q{i}') for i in range(1, N_par_ele+1)])
    a0, a1, c0, c1, c2, c3 = sp.symbols('a0 a1 c0 c1 c2 c3')
    A_e = sp.Symbol('A_e')
    E_e = sp.Symbol('E_e')
    J_e = sp.Symbol('J_e')
    ro_e = sp.Symbol('ro_e')
    T = sp.Symbol('T')
    fo_E = sp.Symbol('fo_E')
    Qglo_pel_curr1_mode, Qglo_pel_curr2_mode, Qglo_pel_curr3_mode, Qglo_pel_curr4_mode, Qglo_pel_curr5_mode, Qglo_pel_curr6_mode= sp.symbols('Qglo_pel_curr1_mode Qglo_pel_curr2_mode Qglo_pel_curr3_mode Qglo_pel_curr4_mode Qglo_pel_curr5_mode Qglo_pel_curr6_mode')
    beta_curr, f1, f0, x, g3, g2, g1, g0, h_e = sp.symbols('beta_curr f1 f0 x g3 g2 g1 g0 h_e')
    X_old, Y_old = sp.symbols('X_old Y_old')

    return x, h_e, beta_e, qe, a0, a1, c0, c1, c2, c3, A_e, E_e, J_e, ro_e, T, fo_E, Qglo_pel_curr1_mode, Qglo_pel_curr2_mode, Qglo_pel_curr3_mode, Qglo_pel_curr4_mode, Qglo_pel_curr5_mode, Qglo_pel_curr6_mode, beta_curr, f1, f0, x, g3, g2, g1, g0, h_e, X_old, Y_old


def calculate_energies(x, qe, h_e, beta_e, E_e, J_e, A_e, ro_e, chi_beam, eps_beam, ve_beam, ue_beam):
    # Calculate potential energy (Pot_beam) and kinetic energy (Kin_beam)
    Pot_beam = 1 / 2 * sp.integrate(E_e * J_e * chi_beam**2 + E_e * A_e * eps_beam**2, (x, 0, h_e))
    Kin_beam = 1 / 2 * ro_e * A_e * sp.integrate(ve_beam**2 + ue_beam**2, (x, 0, h_e))

    return Pot_beam, Kin_beam


def calculate_beam_displacement_equations(x, h_e, beta_e, qe, a0, a1, c0, c1, c2, c3):
    # Compute v1, u1, v2, u2
    v1 = -qe[0] * sp.sin(beta_e) + qe[1] * sp.cos(beta_e)
    u1 = qe[0] * sp.cos(beta_e) + qe[1] * sp.sin(beta_e)
    v2 = -qe[3] * sp.sin(beta_e) + qe[4] * sp.cos(beta_e)
    u2 = qe[3] * sp.cos(beta_e) + qe[4] * sp.sin(beta_e)

    # Define beam displacement equations
    u_beam = a0 + a1 * x
    v_beam = c0 + c1 * x + c2 * x**2 + c3 * x**3

    # Define equilibrium equations
    equations = [
        v_beam.subs(x, 0) - v1,
        sp.diff(v_beam, x).subs(x, 0) - qe[2],
        v_beam.subs(x, h_e) - v2,
        sp.diff(v_beam, x).subs(x, h_e) - qe[5],
        u_beam.subs(x, 0) - u1,
        u_beam.subs(x, h_e) - u2
    ]

    # Solve and assign the solution
    sol = sp.solve(equations, (c0, c1, c2, c3, a0, a1))
    ve_beam = v_beam.subs(sol)
    ue_beam = u_beam.subs(sol)

    # Lambdify ve_beam and ue_beam
    ve_beam_func = lambdify((x, qe, h_e, beta_e), ve_beam, "numpy")
    ue_beam_func = lambdify((x, qe, h_e, beta_e), ue_beam, "numpy")

    return ve_beam_func, ue_beam_func, ve_beam, ue_beam

def assemble_global_matrices(int N_par_ele, int N_par_tot, int N_ele_tot, Pot_beam, Kin_beam, qe, h, A, E, J, beta, ro, pel, h_e, A_e, E_e, J_e, beta_e, ro_e):
    K_beam = Matrix.zeros(N_par_ele)
    M_beam = Matrix.zeros(N_par_ele)

    # Compute K_beam and M_beam
    for i in range(N_par_ele):
        for j in range(N_par_ele):
            K_beam[i, j] = sp.diff(sp.diff(Pot_beam, qe[i]), qe[j])
            M_beam[i, j] = sp.diff(sp.diff(Kin_beam, qe[i]), qe[j])

    # Initialize element stiffness matrix (Ke) and global stiffness matrix (K)
    Ke = np.zeros(N_par_ele)
    K = np.zeros((N_par_tot, N_par_tot))
    Me = np.zeros(N_par_ele)
    M = np.zeros((N_par_tot, N_par_tot))

    # Compute Ke, Me and assemble K, M
    for e in range(N_ele_tot):
        Ke = K_beam.subs({h_e: h[e], A_e: A[e], E_e: E[e], J_e: J[e], beta_e: beta[e]})
        Me = M_beam.subs({h_e: h[e], A_e: A[e], E_e: E[e], J_e: J[e], beta_e: beta[e], ro_e: ro[e]})
        for i in range(N_par_ele):
            for j in range(N_par_ele):
                K[pel[e, i]-1, pel[e, j]-1] += Ke[i, j]
                M[pel[e, i]-1, pel[e, j]-1] += Me[i, j]

    return K, M

def apply_boundary_conditions(int N_par_tot, int N_nod_tot, int N_par_nod, w, K, M):
    # Initialize matrix
    W = np.zeros(N_par_tot, dtype=int)
    count = 0

    # Impose boundary conditions
    # W = 1 if the degree of freedom is fixed, 0 if it is free
    for i in range(N_nod_tot):
        for j in range(N_par_nod):
            W[count] = w[i, j]
            count += 1

    # Modify K and M matrices based on W values
    for i in range(N_par_tot):
        if W[i] == 1:
            # Set all elements in the i-th row and column to 0
            K[i, :] = 0
            K[:, i] = 0
            M[i, :] = 0
            M[:, i] = 0

            # Set the diagonal element to 1 or 1e-30
            K[i, i] = 1
            M[i, i] = 1e-30

    return K, M

def compute_eigenvalues_and_eigenvectors(K, M):
    # Compute eigenvalues (lamb) and eigenvectors (phis)
    lamb, phis = np.linalg.eig(np.linalg.inv(M) @ K)

    # Get the indices that would sort lamb in descending order
    idx = np.argsort(lamb)[::-1]

    # Sort lamb and phis
    lamb_r = lamb[idx]
    phis_r = phis[:, idx]

    # Normalize eigenvectors
    N_par_tot = len(lamb)
    phis_norm = np.zeros((N_par_tot, N_par_tot))
    for i in range(N_par_tot):
        c = np.sqrt(np.dot(phis_r[:, i].T, M @ phis_r[:, i]))
        phis_norm[:, i] = phis_r[:, i] / c

    return lamb_r, phis_norm

def get_mode_indices(lamb_r, N_plots):
    # Calculate periods
    period = 2 * np.pi / np.sqrt(lamb_r)

    # Get the indices that would sort the periods in ascending order
    sorted_indices = np.argsort(period)

    # Get the indices of the modes to be plotted
    index_modes = sorted_indices[-N_plots:]

    # The indices are in ascending order of the periods, but we want them in descending order
    index_modes = index_modes[::-1]

    # Sorted periods
    sorted_period = period[np.argsort(period)]

    return index_modes

In [ ]:
#@title Choose which file to analyze
csv_name = "combos/combo6_60.csv"
X_disp_tot = "X_combo_output"
E_combo_output = "E_combo_output"
A_combo_output = "A_combo_output"
J_combo_output = "J_combo_output"
h_combo_output = "h_combo_output"
T_combo_output = "T_combo_output"
w_combo_output = "w_combo_output"

plot_emphasis = 1

# Extract the first number and everything from that number till the end (excluding extension)
match = re.search(r'\d.*?(?=\.)', csv_name)

if match:
    extracted_sequence = match.group()
    X_disp_tot += extracted_sequence
    E_combo_output += extracted_sequence
    A_combo_output += extracted_sequence
    J_combo_output += extracted_sequence
    h_combo_output += extracted_sequence
    T_combo_output += extracted_sequence
    w_combo_output += extracted_sequence

X_output_file = X_disp_tot + ".csv"
E_output_file = E_combo_output + ".csv"
A_output_file = A_combo_output + ".csv"
J_output_file = J_combo_output + ".csv"
h_output_file = h_combo_output + ".csv"
T_output_file = T_combo_output + ".csv"
w_output_file = w_combo_output + ".csv"
X_output_file, E_output_file, A_output_file, J_output_file, h_output_file, T_output_file, w_output_file

('X_combo_output6_60.csv',
 'E_combo_output6_60.csv',
 'A_combo_output6_60.csv',
 'J_combo_output6_60.csv',
 'h_combo_output6_60.csv',
 'T_combo_output6_60.csv',
 'w_combo_output6_60.csv')

In [ ]:
#@title Main function to run
start_time = time.time()
# Read the combo file
modulus_combo = np.loadtxt(csv_name, delimiter=',', skiprows=1)
print(len(modulus_combo))

# -------- FINAL RESULT ARRAYS --------
# Define lists
X_new_sub = np.zeros((N_plots, N_ele_tot, N_discritizations))
Y_new_sub = np.zeros((N_plots, N_ele_tot, N_discritizations))
# Initialize X_disp
X_disp = np.zeros((N_plots, N_ele_tot))
Y_disp = np.zeros((N_plots, N_ele_tot))
# Store the total displacements
X_disp_tot = np.zeros((len(modulus_combo), N_plots, N_ele_tot))
E_output = np.zeros((len(modulus_combo), N_ele_tot))
A_output = np.zeros((len(modulus_combo), N_ele_tot))
J_output = np.zeros((len(modulus_combo), N_ele_tot))
h_output = np.zeros((len(modulus_combo), N_ele_tot))
T_output = np.zeros((len(modulus_combo), N_plots, N_ele_tot))
w_output = np.zeros((len(modulus_combo), N_plots, N_ele_tot))


# -------- INITIAL DATA --------
# Global position of each element
indices = np.arange(1, N_nod_tot + 1)  # array of indices from 1 to N_nod_tot

# Calculate X positions
X = calculate_X_positions(indices, N_columns, X_dist)

# Calculate Y positions
Y = calculate_Y_positions(indices, N_columns, Y_dist)


# -------- BOUNDARY CONDITIONS --------
# For non ordered bounds, must be changed.
w = np.zeros((N_nod_tot, N_par_nod))
w[:N_columns, :] = 1

# Initialize matrices
W = np.zeros(N_par_tot, dtype=int)
count = 0

# W = 1 if the degree of freedom is fixed, 0 if it is free
for i in range(N_nod_tot):
    for j in range(N_par_nod):
        W[count] = w[i, j]
        count += 1


# -------- ELEMENT NODE INDICIES (ele_nod[element,node]) --------
ele_nod = calculate_element_node_indices(N_floors, N_columns)


# -------- PAR MATRIX AND PEL MATRIX --------
# Initialize matrices
par = np.arange(1, N_nod_tot * N_par_nod + 1).reshape(N_nod_tot, N_par_nod).astype(int)
pel = np.zeros((N_ele_tot, 2 * N_par_nod), dtype=int)

# Assign values to pel matrix
for i in range(N_ele_tot):
    pel[i, :N_par_nod] = par[ele_nod[i, 0] - 1, :]
    pel[i, N_par_nod:] = par[ele_nod[i, 1] - 1, :]


# -------- ELEMENT CHARACTERIZING DATA --------
J = np.zeros(N_ele_tot)
A = np.zeros(N_ele_tot)
beta = np.zeros(N_ele_tot)
E = np.array([elastic_mod] * N_ele_tot, dtype=float)
ro = np.array([unit_weight] * N_ele_tot, dtype=float)
col_idx = np.zeros(N_ele_tot, dtype=int)

# Calculate h, length of the elements
h = calculate_element_length(N_ele_tot, N_columns, X_dist, Y_dist)

# Calculate beta
for i in range(N_ele_tot):
    beta[i] = np.arccos((X[ele_nod[i, 1] - 1] - X[ele_nod[i, 0] - 1]) / h[i])


# -------- SYMBOLIC CALCULATIONS (Moved outside loop) --------
# Define symbolic variables
(x, h_e, beta_e, beta_curr, qe, a0, a1, c0, c1, c2, c3, A_e, E_e, J_e, ro_e, T, 
fo_E, Qglo_pel_curr1_mode, Qglo_pel_curr2_mode, Qglo_pel_curr3_mode, 
Qglo_pel_curr4_mode, Qglo_pel_curr5_mode, Qglo_pel_curr6_mode, X_old, Y_old) = initialize_symbols(N_par_ele)

# Calculate the symbolic displacements
ve_beam_func, ue_beam_func, ve_beam, ue_beam  = calculate_beam_displacement_equations(x, h_e, beta_e, qe, a0, a1, c0, c1, c2, c3)

# Calculate potential energy (Pot_beam) and kinetic energy (Kin_beam)
Pot_beam, Kin_beam, chi_beam, eps_beam = calculate_energies(x, qe, h_e, beta_e, E_e, J_e, A_e, ro_e, ve_beam, ue_beam)



# -------- STIFFNESS AND MASS SYMBOLIC MATRICES (MOVED OUT OF LOOP) --------
# Function to compute K_beam and M_beam for a given pair of indices
K_beam = np.zeros((N_par_ele, N_par_ele), dtype=object)
M_beam = np.zeros((N_par_ele, N_par_ele), dtype=object)
def compute_K_M(i, j):
    K_beam[i][j] = sp.lambdify((x, h_e, A_e, E_e, J_e, beta_e, ro_e),
                            sp.diff(sp.diff(Pot_beam, qe[i]), qe[j]), 'numpy')
    M_beam[i][j] = sp.lambdify((x, h_e, A_e, E_e, J_e, beta_e, ro_e),
                            sp.diff(sp.diff(Kin_beam, qe[i]), qe[j]), 'numpy')

# Use ThreadPoolExecutor to parallelize the computation
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(compute_K_M, i, j) for i in range(N_par_ele) for j in range(N_par_ele)]
    # Wait for all computations to complete
    for future in futures:
        future.result()



# -------- GLOBAL DISPLACEMENTS (MOVED OUT OF LOOP) --------
# Defines local displacements and calculates global displacements symbolic lambda functions
X_new_sub_func, Y_new_sub_func = calculate_global_displacements(Qglo_pel_curr1_mode, Qglo_pel_curr2_mode, Qglo_pel_curr3_mode, Qglo_pel_curr4_mode, 
                           Qglo_pel_curr5_mode, Qglo_pel_curr6_mode, beta_curr, h_e)


count = 1
for i in range(1, len(col_idx)):
    if col_idx[i] == 0:
        J[count] = width_beam**3 * height_beam / 12
        A[count] = width_beam * height_beam
    count += 1


for l in range(1):
    for iterr, E_combos in enumerate(modulus_combo):
        # Calculate J, A, and h
        J[0] = width_column * height_column**3 / 12
        A[0] = width_column * height_column
        E[0] = E_combos[0]
        for rl_idx, idx in enumerate(col_idx):
            if idx != 0:
                J[rl_idx] = width_column * height_column**3 / 12
                A[rl_idx] = width_column * height_column
                E[rl_idx] = E_combos[idx]



        # # Define external force array (fo)
        # fo = sp.zeros(N_ele_tot, 1)
        # for i in range(0, N_ele_tot, 3):
        #     fo[i] = -po * sp.sin(theta * T)

        # # Calculate external force (ext_f)
        # ext_f = sp.integrate(fo_E * ve_beam, (x, 0, h_e))

        # # Initialize external force vector (p_E)
        # p_E = Matrix.zeros(N_par_tot, 1)
        # for i in range(N_par_ele):
        #     p_E[i] = sp.diff(ext_f, qe[i]).evalf()

        # # Initialize global force vector (P)
        # P = np.zeros(N_par_tot)


        # -------- ASSEMBLE GLOBAL STIFFNESS MATRIX --------
        # Initialize K and M
        K = np.zeros((N_par_tot, N_par_tot))
        M = np.zeros((N_par_tot, N_par_tot))

        # Compute Ke, Me and assemble K, M using NumPy operations
        for e in range(N_ele_tot):
            for i in range(N_par_ele):
                for j in range(N_par_ele):
                    K[pel[e, i]-1, pel[e, j]-1] += K_beam[i, j](0, h[e], A[e], E[e], J[e], beta[e], ro[e])
                    M[pel[e, i]-1, pel[e, j]-1] += M_beam[i, j](0, h[e], A[e], E[e], J[e], beta[e], ro[e])


        K, M = apply_boundary_conditions(N_par_tot, N_nod_tot, N_par_nod, W, K, M)



        # -------- EIGENVALUE PROBLEM --------
        # Compute eigenvalues (lamb) and eigenvectors (phis)
        # Already taken the real part and normalized
        lamb_r, phis_norm = compute_eigenvalues_and_eigenvectors(K, M)


        # Calculate periods to get the top contributing index_modes
        index_modes, period = get_mode_indices(lamb_r, phis_norm, N_plots)

        # Extract lambdas and corresponding eigenvectors
        lamb_plots = lamb_r[index_modes]
        phis_plots = phis_norm[:, index_modes]


        # Iterate over N_plots and N_ele_tot
        for j in range(N_plots):
            for e in range(N_ele_tot):
                # Substitute values into X_new and Y_new using the lambda functions
                substitutions = {
                    Qglo_pel_curr1_mode: phis_plots[pel[e, 0] - 1, j]*plot_emphasis,
                    Qglo_pel_curr2_mode: phis_plots[pel[e, 1] - 1, j]*plot_emphasis,
                    Qglo_pel_curr3_mode: phis_plots[pel[e, 2] - 1, j]*plot_emphasis,
                    Qglo_pel_curr4_mode: phis_plots[pel[e, 3] - 1, j]*plot_emphasis,
                    Qglo_pel_curr5_mode: phis_plots[pel[e, 4] - 1, j]*plot_emphasis,
                    Qglo_pel_curr6_mode: phis_plots[pel[e, 5] - 1, j]*plot_emphasis,
                    X_old: X[ele_nod[e, 0] - 1],
                    Y_old: Y[ele_nod[e, 0] - 1],
                    beta_curr: beta[e],
                    h_e: h[e],
                }

                # Save the displacement [# of element, # of mode]
                X_new_sub[j][e] = X_new_sub_func(np.linspace(0, h[e],
                                                            N_discritizations), **{str(k): v for k, v in substitutions.items()})
                X_disp[j, e] = X_new_sub[j][e][-1] - X[ele_nod[e, 1]-1]

                # Y_new_sub[j][e] = Y_new_sub_func(np.linspace(0, h[e],
                #                                              N_discritizations), **{str(k): v for k, v in substitutions.items()})
                # Y_disp[j, e] = Y_new_sub[j][e][-1] - Y[ele_nod[e, 1]-1]

                # print(f"Mode: {j}, Element: {e}, X_disp: {X_disp[j, e]}")


                plt.plot(X_new_sub[j][e], Y_new_sub[j][e])  # Plot elements for the current mode
            plt.axis('equal')  # Make x and y axes have the same scale
            plt.show()  # Display the plot for the current mode

        X_disp_tot[iterr] = X_disp
        E_output[iterr] = E
        A_output[iterr] = A
        J_output[iterr] = J
        h_output[iterr] = h
        for i in range(N_plots):
          T_output[iterr][i] = np.full(N_ele_tot, period[index_modes[i]])
          w_output[iterr][i] = np.full(N_ele_tot, math.sqrt(lamb_plots[i]))

        # print(X_disp[0,:])
end_time = time.time()
print("Elapsed time:", end_time - start_time, "seconds")

np.savetxt(X_output_file, X_disp_tot.reshape(len(modulus_combo), -1), delimiter=",")
np.savetxt(E_output_file, E_output, delimiter=",")
np.savetxt(A_output_file, A_output, delimiter=",")
np.savetxt(J_output_file, J_output, delimiter=",")
np.savetxt(h_output_file, h_output, delimiter=",")
np.savetxt(T_output_file, T_output.reshape(len(modulus_combo), -1), delimiter=",")
np.savetxt(w_output_file, w_output.reshape(len(modulus_combo), -1), delimiter=",")

91854
Elapsed time: 811.9795372486115 seconds


In [ ]:
files.download(X_output_file)
files.download(E_output_file)
files.download(A_output_file)
files.download(J_output_file)
files.download(h_output_file)
files.download(T_output_file)
files.download(w_output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>